## LSTM 2

### Imports

In [5]:
import pandas as pd
import os
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Bidirectional
from keras.preprocessing import sequence
from keras.layers import Dropout
import h5py
import utility_functions as uf
from keras.models import model_from_json
from keras.models import load_model
from nltk.tokenize import RegexpTokenizer

### Load data

In [2]:
df1 = pd.read_json("./data/Sarcasm_Headlines_Dataset.json", lines=True)
df2 = pd.read_json("./data/Sarcasm_Headlines_Dataset_v2.json", lines=True)
# re-order attibute columns in df2
df2 = df2[['article_link','headline','is_sarcastic']]
df = pd.concat([df1, df2], axis=0)
df = df.drop(['article_link'], axis=1)
print(len(df))
df.head()

55328


,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [ ]:
def load_data_all(data_dir, all_data_path,pred_path, gloveFile, first_run, load_all):

    # Load embeddings for the filtered glove list
    if load_all == True:
        weight_matrix, word_idx = uf.load_embeddings(gloveFile)
    else:
        weight_matrix, word_idx = uf.load_embeddings(filtered_glove_path)

    len(word_idx)
    len(weight_matrix)

    #%%
    # create test, validation and trainng data
    all_data = uf.read_data(all_data_path)
    train_data, test_data, dev_data = uf.training_data_split(all_data, 0.8, data_dir)

    train_data = train_data.reset_index()
    dev_data = dev_data.reset_index()
    test_data = test_data.reset_index()

    #%%
    # inputs from dl_sentiment that are hard coded but need to be automated
    maxSeqLength, avg_words, sequence_length = uf.maxSeqLen(all_data)
    numClasses = 10
    #%%

     # load Training data matrix
    train_x = uf.tf_data_pipeline_nltk(train_data, word_idx, weight_matrix, maxSeqLength)
    test_x = uf.tf_data_pipeline_nltk(test_data, word_idx, weight_matrix, maxSeqLength)
    val_x = uf.tf_data_pipeline_nltk(dev_data, word_idx, weight_matrix, maxSeqLength)

    #%%
    # load labels data matrix
    train_y = uf.labels_matrix(train_data)
    val_y = uf.labels_matrix(dev_data)
    test_y = uf.labels_matrix(test_data)


     #%%

    # summarize size
    print("Training data: ")
    print(train_x.shape)
    print(train_y.shape)

    # Summarize number of classes
    print("Classes: ")
    print(np.unique(train_y.shape[1]))

    return train_x, train_y, test_x, test_y, val_x, val_y, weight_matrix, word_idx



In [ ]:
def main():

    max_words = 56 # max no of words in your training data
    batch_size = 2000 # batch size for training
    EMBEDDING_DIM = 100 # size of the word embeddings
    train_flag = True # set True if in training mode else False if in prediction mode

    if train_flag:
        # create training, validation and test data sets
        # load the dataset
        path = '.'
        data_dir = path+'/Data'
        all_data_path = path+'/Data/'
        pred_path = path+'/Data/output_model/test_pred.csv'
        gloveFile = path+'/embeddings/glove_6B_100d.txt'
        first_run = False
        load_all = True
        
        # Split the data into train, val and test sets. Also the corresponding embeddings for the data are stored in the weight_matrix variable.
        train_x, train_y, test_x, test_y, val_x, val_y, weight_matrix, word_idx = load_data_all(data_dir, all_data_path,pred_path, gloveFile, first_run, load_all)
        
        # create model strucutre
        model = create_model_rnn(weight_matrix, max_words, EMBEDDING_DIM)

        # train the model
        trained_model =train_model(model,train_x, train_y, test_x, test_y, val_x, val_y, batch_size, path)   # run model live


        # serialize weights to HDF5
        model.save_weights(path+"/model/best_model.h5")
        print("Saved model to disk")

    else:
        gloveFile = path+'/Data/glove/glove_6B_100d.txt'
        weight_matrix, word_idx = uf.load_embeddings(gloveFile)
        weight_path = path +'/model/best_weights_bi_glove.hdf5'
        loaded_model = load_model(weight_path)
        loaded_model.summary()
        data_sample = "Great!! it is raining today!!"
        result = live_test(loaded_model,data_sample, word_idx)
        print (result)
